In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)
%tensorflow_version 1.x

In [ ]:
!pip install tiffile


!pip install gputools


!pip install vollseg
!pip install n2v

In [ ]:

# We import all our dependencies.
from n2v.models import N2VConfig, N2V
import numpy as np
import os

from csbdeep.utils import plot_history,plot_some
from n2v.utils.n2v_utils import manipulate_val_data
from n2v.internals.N2V_DataGenerator import N2V_DataGenerator
from matplotlib import pyplot as plt
import urllib
import os
import zipfile

In [ ]:
BaseDir = '/content/drive/My Drive/data/'

Model_Dir = '/content/drive/My Drive/data/'
Model_Name = 'ScipyDenoising'
datagen = N2V_DataGenerator()

imgs = datagen.load_imgs_from_directory(directory = BaseDir + "Raw/", dims='ZYX')
plt.figure(figsize=(32,16))
plt.imshow(np.max(imgs[0][0,...,0],axis=0), 
           cmap='magma',
           vmin=np.percentile(imgs[0],0.1),
           vmax=np.percentile(imgs[0],99.9)
          )
plt.show()





In [ ]:
patch_shape = (16, 128, 128)
Epochs = 200
NetworkDepth = 3
batch_size = 1
patches = datagen.generate_patches_from_list(imgs[:1], shape=patch_shape)
print(patches.shape)
X = patches[:120]
X_val = patches[120:]

# Let's look at two patches.
plt.figure(figsize=(14,7))
plt.subplot(1,2,1)
plt.imshow(X[0,4,...,0],cmap='magma')
plt.title('Training Patch');
plt.subplot(1,2,2)
plt.imshow(X_val[0,4,...,0],cmap='magma')
plt.title('Validation Patch');

In [ ]:
config = N2VConfig(X, unet_kern_size=3, unet_n_depth = NetworkDepth,
                   train_steps_per_epoch=400,train_epochs= Epochs, train_loss='mse', batch_norm=True, 
                   train_batch_size=batch_size, n2v_perc_pix=0.198,n2v_patch_shape= patch_shape, 
                   n2v_manipulator='uniform_withCP', n2v_neighborhood_radius=4)

# Let's look at the parameters stored in the config-object.
vars(config)
model = N2V(config=config, name=Model_Name, basedir=BaseDir)

In [ ]:

history = model.train(X, X_val)

In [ ]:
print(sorted(list(history.history.keys())))
plt.figure(figsize=(16,5))
plot_history(history,['loss','val_loss']);

In [ ]:
plt.figure(figsize=(25,7))
_P = model.keras_model.predict(X_val[:5])

plot_some(X_val[:5],_P,pmax=99.5)
plt.suptitle('5 example validation patches\n'      
             'top row: input (source),  '          
             'middle row: denoised),  '
             );